In [27]:
import argparse
import json
import pandas as pd
import numpy as np

import gzip
import io

In [4]:
def extract_ids(fasta_file):
    """Extract Accession IDs (entry names) from a fasta file

    Parameters
    ----------
    fasta_file: str

    Returns
    -------
    out: list of tuples, (Accession ID, date)

    """

    out = []

    # Read sequences
    cur_entry = ""
    cur_seq = ""

    # Get the date from the fasta file name, as a string
    file_date = Path(fasta_file).name.replace(".fa.gz", "")

    with gzip.open(fasta_file, "rt") as fp:
        lines = fp.readlines()
        for i, line in enumerate(lines):
            # Strip whitespace
            line = line.strip()

            # Ignore empty lines that aren't the last line
            if not line and i < (len(lines) - 1):
                continue

            # If not the name of an entry, add this line to the current sequence
            # (some FASTA files will have multiple lines per sequence)
            if line[0] != ">":
                cur_seq = cur_seq + line

            # Start of another entry = end of the previous entry
            if line[0] == ">" or i == (len(lines) - 1):
                # Avoid capturing the first one and pushing an empty sequence
                if cur_entry:
                    out.append((cur_entry, file_date,))

                # Clear the entry and sequence
                cur_entry = line[1:]
                # Ignore anything past the first whitespace
                if cur_entry:
                    cur_entry = cur_entry.split()[0]
                cur_seq = ""

    # print("Read {} entries for file {}".format(len(out), fasta_file))

    return out


In [5]:
from pathlib import Path

In [18]:
# processed_fasta_files,
# mutation_files,
# Mutations must occur at least this many times to pass filters
count_threshold=3,
mode="dna"  # dna, gene_aa, protein_aa

processed_fasta_files = Path('/Users/atchen/covidcg_rsv/data_genbank_rsv/fasta_processed')
mutation_files = sorted(Path('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation').glob('*.csv'))
# mutation_files = [str(f) for f in mutation_files]
mutation_files[:5]


[PosixPath('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation/A_1998-09_dna_mutation.csv'),
 PosixPath('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation/A_2006-07_dna_mutation.csv'),
 PosixPath('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation/A_2007-11_dna_mutation.csv'),
 PosixPath('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation/A_2008-03_dna_mutation.csv'),
 PosixPath('/Users/atchen/covidcg_rsv/data_genbank_rsv/dna_mutation/A_2008-06_dna_mutation.csv')]

In [8]:
manifest = []
for fasta_file in sorted(Path(processed_fasta_files).glob("*.fa.gz")):
    manifest.extend(extract_ids(fasta_file))
manifest = pd.DataFrame.from_records(manifest, columns=["Accession ID", "date"])
pruned_manifest = manifest.drop_duplicates(["Accession ID"], keep="last")

In [9]:
pruned_manifest

,Accession ID,date
0,AF086899,A_1998-09
1,AF086868,A_1998-09
2,AF086869,A_1998-09
3,AF086870,A_1998-09
4,AF086871,A_1998-09
...,...,...
24190,MW678572,B_2022-03
24191,MW678573,B_2022-03
24192,MW678571,B_2022-03
24193,LC699635,B_2022-03


In [32]:
# Add references to manifest
pruned_manifest['reference'] = np.nan
pruned_manifest['reference'] = pruned_manifest['reference'].apply(lambda x: ['NC_038235.1', 'NC_001781.1', 'KX858757.1', 'KX858756.1'])
pruned_manifest = pruned_manifest.explode('reference')
pruned_manifest.head()


,Accession ID,date,reference
0,AF086899,A_1998-09,NC_038235.1
0,AF086899,A_1998-09,NC_001781.1
0,AF086899,A_1998-09,KX858757.1
0,AF086899,A_1998-09,KX858756.1
1,AF086868,A_1998-09,NC_038235.1


In [33]:
# Dump all mutation chunks into a text buffer
mutation_df_io = io.StringIO()
for i, chunk in enumerate(mutation_files):
    file_date = Path(chunk).name.replace("_" + mode + "_mutation.csv", "")
    with open(chunk, "r") as fp_in:
        # Write dates, so we can remove duplicate sequences
        # and default to the mutations of the latest sequence, by date
        for j, line in enumerate(fp_in):
            # Write the header of the first file
            if i == 0 and j == 0:
                mutation_df_io.write(line.strip() + ",date\n")
            # Or write any line that's not the header
            # (to avoid writing the header more than once)
            elif j > 0:
                mutation_df_io.write(line.strip() + "," + file_date + "\n")

In [34]:
# Read the buffer into a dataframe, then discard the buffer
mutation_df_io.seek(0)
mutation_df = pd.read_csv(mutation_df_io)
mutation_df_io.close()

In [35]:
mutation_df.head()

,reference,Accession ID,pos,ref,alt,date
0,NC_038235.1,AF086869,5427,T,C,A_1998-09
1,NC_038235.1,AF086869,5429,C,T,A_1998-09
2,NC_038235.1,AF086869,5446,A,G,A_1998-09
3,NC_038235.1,AF086869,5447,G,A,A_1998-09
4,NC_038235.1,AF086869,5503,C,T,A_1998-09


In [36]:
mutation_df = pruned_manifest.set_index(["Accession ID", 'reference', "date"]).join(
    mutation_df.set_index(["Accession ID", 'reference', "date"]), how="left"
)
mutation_df.reset_index(inplace=True)
mutation_df.head()

,Accession ID,reference,date,pos,ref,alt
0,AB470478,KX858756.1,A_2009-05,NaN,NaN,NaN
1,AB470478,KX858757.1,A_2009-05,NaN,NaN,NaN
2,AB470478,NC_001781.1,A_2009-05,NaN,NaN,NaN
3,AB470478,NC_038235.1,A_2009-05,5347.0,T,A
4,AB470478,NC_038235.1,A_2009-05,5351.0,T,C


In [39]:
no_mutation_seqs = mutation_df.loc[mutation_df["pos"].isna(), ["Accession ID", "reference"]]
no_mutation_seqs.head()

,Accession ID,reference
0,AB470478,KX858756.1
1,AB470478,KX858757.1
2,AB470478,NC_001781.1
29,AB470479,KX858756.1
30,AB470479,KX858757.1


In [40]:
# Remove sequences with no mutations before counting mutations
mutation_df = mutation_df.loc[~mutation_df["pos"].isna()]
mutation_df.loc[:, "pos"] = mutation_df["pos"].astype(int)
# Replace NaNs in the 'ref' and 'alt' column with '-'
mutation_df.fillna("-", inplace=True)
mutation_df.head()

,Accession ID,reference,date,pos,ref,alt
3,AB470478,NC_038235.1,A_2009-05,5347,T,A
4,AB470478,NC_038235.1,A_2009-05,5351,T,C
5,AB470478,NC_038235.1,A_2009-05,5364,C,T
6,AB470478,NC_038235.1,A_2009-05,5384,G,A
7,AB470478,NC_038235.1,A_2009-05,5396,A,G


In [52]:
groupby_cols = ["pos", "ref", "alt"]
if mode == "gene_aa":
    groupby_cols.insert(0, "gene")
elif mode == "protein_aa":
    groupby_cols.insert(0, "protein")

In [53]:
# Generate mutation strings for initial dataframes
# e.g., for DNA, its pos|ref|alt
# mutation_df['mutation_str'] = np.nan
mutation_df = mutation_df.assign(
    mutation_str=(
        mutation_df[groupby_cols[0]]
        .astype(str)
        .str.cat(
            [mutation_df[col].astype(str) for col in groupby_cols[1:]], sep="|"
        )
    )
)
mutation_df.head()

,Accession ID,reference,date,pos,ref,alt,mutation_str,count,mutation_id
1513401,KJ627672,KX858756.1,A_2014-04,1000,T,A,1000|T|A,10,0
1517050,KJ627673,KX858756.1,A_2014-04,1000,T,A,1000|T|A,10,0
1520685,KJ627674,KX858756.1,A_2014-04,1000,T,A,1000|T|A,10,0
1524318,KJ627675,KX858756.1,A_2014-04,1000,T,A,1000|T|A,10,0
1571233,KJ627688,KX858756.1,A_2014-04,1000,T,A,1000|T|A,10,0


In [54]:
# Collapse by Accession ID and count occurrences
# Filter out low global occurrence mutations
valid_mutations = mutation_df.groupby("mutation_str")["Accession ID"].count()
valid_mutations = valid_mutations[valid_mutations > count_threshold]
valid_mutations

mutation_str
10000|T|C               23
10000|T|G               12
10003|C|T                4
10005|T|A             1973
10006|T|A             1557
                      ... 
99|-|CCTTTGGTTTGAG       4
99|-|GTGTTTGAG           5
99|C|T                2232
99|T|C                1841
9|G|A                    7
Name: Accession ID, Length: 24637, dtype: int64

In [57]:
# Filter mutations by valid mutations
print(len(mutation_df))
mutation_df = mutation_df.drop(columns=['count']).join(
    valid_mutations.rename("count"), how="right", on="mutation_str"
)
print(len(mutation_df))


13852674
13852674


In [58]:
mutation_df.head()

,Accession ID,reference,date,pos,ref,alt,mutation_str,mutation_id,count
2516527,KJ723479,NC_038235.1,B_2014-05,10000,T,C,10000|T|C,14533,23
2937862,KP258718,NC_038235.1,B_2014-12,10000,T,C,10000|T|C,14533,23
2998883,KP258736,NC_038235.1,B_2014-12,10000,T,C,10000|T|C,14533,23
3174349,KP856963,NC_038235.1,B_2015-04,10000,T,C,10000|T|C,14533,23
3292789,KU316092,NC_038235.1,A_2016-01,10000,T,C,10000|T|C,14533,23


In [59]:
# Map mutations to integer IDs
mutation_map = pd.Series(mutation_df["mutation_str"].unique())
# Flip index and values
mutation_map = pd.Series(mutation_map.index.values, index=mutation_map)
mutation_df["mutation_id"] = mutation_df["mutation_str"].map(mutation_map)
mutation_df.head()

,Accession ID,reference,date,pos,ref,alt,mutation_str,mutation_id,count
2516527,KJ723479,NC_038235.1,B_2014-05,10000,T,C,10000|T|C,0,23
2937862,KP258718,NC_038235.1,B_2014-12,10000,T,C,10000|T|C,0,23
2998883,KP258736,NC_038235.1,B_2014-12,10000,T,C,10000|T|C,0,23
3174349,KP856963,NC_038235.1,B_2015-04,10000,T,C,10000|T|C,0,23
3292789,KU316092,NC_038235.1,A_2016-01,10000,T,C,10000|T|C,0,23


In [90]:
mutation_group_df = (
    mutation_df.groupby(["Accession ID", "reference"], as_index=False)["mutation_id"].agg(list)
)
mutation_group_df.head()

,Accession ID,reference,mutation_id
0,AB470478,NC_038235.1,"[14859, 14894, 14988, 15114, 15179, 15427, 155..."
1,AB470479,NC_038235.1,"[14859, 14894, 14988, 15114, 15427, 15505, 156..."
2,AB470480,NC_001781.1,"[14741, 14840, 14920, 14936, 15040, 15064, 152..."
3,AB470481,NC_001781.1,"[14840, 14921, 14924, 15040, 15064, 15067, 152..."
4,AB470482,NC_001781.1,"[14840, 14921, 14924, 15002, 15040, 15064, 150..."


In [91]:
mutation_group_df.loc[mutation_group_df['mutation_id'].apply(len) == 0]

,Accession ID,reference,mutation_id


In [92]:
mutation_group_df.loc[mutation_group_df['Accession ID'] == 'KJ627672']

,Accession ID,reference,mutation_id
6570,KJ627672,KX858756.1,"[10, 144, 182, 224, 465, 669, 714, 779, 808, 8..."
6571,KJ627672,KX858757.1,"[10, 45, 669, 714, 747, 990, 1223, 1254, 1719,..."
6572,KJ627672,NC_001781.1,"[3, 7, 20, 28, 29, 37, 38, 47, 50, 65, 67, 78,..."
6573,KJ627672,NC_038235.1,"[9, 15, 24, 30, 70, 162, 175, 237, 243, 248, 2..."


In [93]:
mutation_group_df.loc[mutation_group_df['Accession ID'] == 'AB470478']

,Accession ID,reference,mutation_id
0,AB470478,NC_038235.1,"[14859, 14894, 14988, 15114, 15179, 15427, 155..."


In [94]:
no_mutation_seqs.loc[no_mutation_seqs['Accession ID'] == 'AB470478']

,Accession ID,reference
0,AB470478,KX858756.1
1,AB470478,KX858757.1
2,AB470478,NC_001781.1


In [95]:
# Add back the sequences with no mutations
mutation_group_df = (
    pd.concat(
        [
            mutation_group_df,
            no_mutation_seqs.assign(
                mutation_id=[[]] * len(no_mutation_seqs)
            ),
        ],
        axis=0,
        ignore_index=True,
    )
    .sort_values(['Accession ID', 'reference'])
    .reset_index(drop=True)
    #.set_index(["Accession ID", "reference"])
)
mutation_group_df.head()

,Accession ID,reference,mutation_id
0,AB470478,KX858756.1,[]
1,AB470478,KX858757.1,[]
2,AB470478,NC_001781.1,[]
3,AB470478,NC_038235.1,"[14859, 14894, 14988, 15114, 15179, 15427, 155..."
4,AB470479,KX858756.1,[]
